In [22]:
cd /Users/sid/24/refcount/clean/1-runs/run-2023-03-29--00-16---macbook
echo "pwd: $(pwd)"
DATE=$(date)
echo "date: $DATE"
MACHINE=$(uname -a)
echo "machine: $MACHINE"
echo "git status: $(git status --short)"
echo "git commit: $(git rev-parse HEAD)"
ROOT=$(git rev-parse --show-toplevel)
echo "root folder: $ROOT"
OUTFOLDER=$(pwd)
echo "out folder: $OUTFOLDER"

if [ "$(uname -s)" = "Darwin" ]; then
    TIME="gtime"
else
    TIME="command time"
fi
echo "time: $TIME"
$TIME -v echo "time"

pwd: /Users/sid/24/refcount/clean/1-runs/run-2023-03-29--00-16---macbook
date: Fri 29 Mar 2024 01:47:08 GMT
machine: Darwin S47-0K2163M7 23.1.0 Darwin Kernel Version 23.1.0: Mon Oct  9 21:28:31 PDT 2023; root:xnu-10002.41.9~6/RELEASE_ARM64_T8112 arm64
git status: ?? ./
git commit: 18fbdf4993d228bd8bc71d1462a287a581c33a41
root folder: /Users/sid/24/refcount/clean
out folder: /Users/sid/24/refcount/clean/1-runs/run-2023-03-29--00-16---macbook
time: gtime
time
	Command being timed: "echo time"
	User time (seconds): 0.00
	System time (seconds): 0.00
	Percent of CPU this job got: 57%
	Elapsed (wall clock) time (h:mm:ss or m:ss): 0:00.00
	Average shared text size (kbytes): 0
	Average unshared data size (kbytes): 0
	Average stack size (kbytes): 0
	Average total size (kbytes): 0
	Maximum resident set size (kbytes): 1200
	Average resident set size (kbytes): 0
	Major (requiring I/O) page faults: 2
	Minor (reclaiming a frame) page faults: 189
	Voluntary context switches: 1
	Involuntary context sw

## Benchmarking Strategy
- Bench on MacOS.
- Disable RUNTIME_STATS
- Bench stage2, stage3.

In [23]:
rm -rf $ROOT/build/noreuse || true
mkdir -p $ROOT/build/noreuse
cd $ROOT/build/noreuse

# == NO REUSE ==

echo "@@@ NOREUSE BUILD @@@"

CSVNAME=$(date +'%s---%d-%m-%Y---%H:%M:%S')
echo "CSV name is: $CSVNAME"
echo "output file is: OUTFOLDER/$CSVNAME"

cmake ../../ \
  -DCCACHE=OFF \
  -DRUNTIME_STATS=OFF \
  -DLEAN_RESEARCH_IS_REUSE_ACROSS_CONSTRUCTORS_ENABLED=0 \
  -DLEAN_RESEARCH_COMPILER_PROFILE_CSV_PATH=$OUTFOLDER/$CSVNAME

make -j stage1 2>1 >> log.txt
make -j stage2 2>1 >> log.txt
touch ../../src/Init/Prelude.lean
touch $OUTFOLDER/$CSVNAME && echo "" > $OUTFOLDER/$CSVNAME
$TIME -v make -j stage2 &> "time-noreuse-stage2.txt"
mv $OUTFOLDER/$CSVNAME $OUTFOLDER/$CSVNAME.noreuse.stage2.csv
$TIME -v make -j stage3 &> "time-noreuse-stage3.txt"
mv $OUTFOLDER/$CSVNAME $OUTFOLDER/$CSVNAME.noreuse.stage3.csv

curl -d "Done[noreuse]\n*machine:$(uname -a)\n*folder:$OUTFOLDER\n*machine:$MACHINE" ntfy.sh/xISSztEV8EoOchM2
# == REUSE ==

echo "@@@ REUSE BUILD @@@"

rm -rf $ROOT/build/reuse || true
mkdir -p $ROOT/build/reuse
cd $ROOT/build/reuse

CSVNAME="$(date +'%s---%d-%m-%Y---%H:%M:%S')"
OUTFILE="$OUTFOLDER/$CSVNAME"
echo "output file is: $OUTFILE"

cmake ../../ \
  -DCCACHE=OFF \
  -DRUNTIME_STATS=OFF \
  -DLEAN_RESEARCH_IS_REUSE_ACROSS_CONSTRUCTORS_ENABLED=1 \
  -DLEAN_RESEARCH_COMPILER_PROFILE_CSV_PATH=$$OUTFOLDER/$CSVNAME
make -j stage1  2>1 >> log.txt
make -j stage2  2>1 >> log.txt
touch ../../src/Init/Prelude.lean
touch $OUTFOLDER/$CSVNAME && echo "" > $OUTFOLDER/$CSVNAME
$TIME -v make -j stage2 &> "time-reuse-stage2.txt" # TODO: print to outfolder
mv $OUTFOLDER/$CSVNAME $OUTFOLDER/$CSVNAME.reuse.stage2.csv

$TIME -v make -j stage3 &> "time-reuse-stage3.txt"
mv $OUTFOLDER/$CSVNAME $OUTFOLDER/$CSVNAME.reuse.stage3.csv

echo "DONE!"
curl -d "Done fully\n*machine:$(uname -a)\n*folder:$OUTFOLDER\n*machine:$MACHINE" ntfy.sh/xISSztEV8EoOchM2

@@@ NOREUSE BUILD @@@
CSV name is: 1711676843---29-03-2024---01:47:23
output file is: OUTFOLDER/1711676843---29-03-2024---01:47:23
-- The CXX compiler identification is AppleClang 15.0.0.15000100
-- The C compiler identification is AppleClang 15.0.0.15000100
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /Library/Developer/CommandLineTools/usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /Library/Developer/CommandLineTools/usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Configuring done (1.0s)
-- Generating done (0.0s)
-- Build files have been written to: /Users/sid/24/refcount/clean/build/noreuse
{"id":"v2kagA2BoYHU","time":1711679321,"expires":1711722521,"event":"message","topic":"xISSztEV8EoOchM2","message":"Done[noreuse]

In [24]:
# tail -f $ROOT/build/reuse/time-noreuse-stage2.txt -n 35
# tail -f $ROOT/build/noreuse/time-noreuse-stage2.txt -n 35
cp $ROOT/build/noreuse/time-noreuse-stage2.txt  $OUTFOLDER/time-noreuse-stage2.txt
cp $ROOT/build/noreuse/time-noreuse-stage3.txt  $OUTFOLDER/time-noreuse-stage3.txt

cp $ROOT/build/reuse/time-reuse-stage2.txt  $OUTFOLDER/time-reuse-stage2.txt  # TODO: fix this typo!
cp $ROOT/build/reuse/time-noreuse-stage3.txt  $OUTFOLDER/time-reuse-stage3.txt